<a href="https://www.kaggle.com/code/zaneselvans/openmod-pudl-demo-draft?scriptVersionId=145543868" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import sys

print(f"Python version: {sys.version}")
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlalchemy as sa

print(f"{np.__version__=}")
print(f"{pd.__version__=}")
print(f"{sa.__version__=}")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in sorted(filenames):
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Python version: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
np.__version__='1.23.5'
pd.__version__='2.0.3'
sa.__version__='2.0.17'
/kaggle/input/catalyst-cooperative-pudl/censusdp1tract.sqlite
/kaggle/input/catalyst-cooperative-pudl/ferc1.sqlite
/kaggle/input/catalyst-cooperative-pudl/ferc1_xbrl.sqlite
/kaggle/input/catalyst-cooperative-pudl/ferc1_xbrl_datapackage.json
/kaggle/input/catalyst-cooperative-pudl/ferc1_xbrl_taxonomy_metadata.json
/kaggle/input/catalyst-cooperative-pudl/hourly_emissions_epacems.parquet
/kaggle/input/catalyst-cooperative-pudl/pudl.sqlite


# Create a PUDL DB connection

In [2]:
pudl_engine = sa.create_engine("sqlite:////kaggle/input/catalyst-cooperative-pudl/pudl.sqlite")

In [3]:
frc = pd.read_sql("denorm_fuel_receipts_costs_eia923", pudl_engine).convert_dtypes()
frc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631975 entries, 0 to 631974
Data columns (total 36 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   report_date                              631975 non-null  datetime64[ns]
 1   plant_id_eia                             631975 non-null  Int64         
 2   plant_id_pudl                            631975 non-null  Int64         
 3   plant_name_eia                           631975 non-null  string        
 4   utility_id_eia                           631975 non-null  Int64         
 5   utility_id_pudl                          631975 non-null  Int64         
 6   utility_name_eia                         631975 non-null  string        
 7   state                                    631975 non-null  string        
 8   contract_type_code                       631737 non-null  string        
 9   contract_expiration_date  

In [4]:
frc.sample(20)

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,state,contract_type_code,contract_expiration_date,...,natural_gas_transport_code,natural_gas_delivery_contract_type_code,moisture_content_pct,chlorine_content_ppm,data_maturity,mine_id_msha,mine_name,mine_state,coalmine_county_id_fips,mine_type_code
8783,2008-02-01,10866,3401,Peoria,772,522,Archer Daniels Midland Co,IL,C,2008-12-01,...,firm,<NA>,<NA>,<NA>,final,<NA>,<NA>,<NA>,<NA>,<NA>
262608,2012-08-01,55051,4326,Plaquemine,7191,560,Georgia Gulf Corp,LA,S,NaT,...,firm,<NA>,<NA>,<NA>,final,<NA>,<NA>,<NA>,<NA>,<NA>
630574,2022-12-01,7848,2892,Holden,924,539,Associated Electric Coop Inc,MO,S,NaT,...,interruptible,interruptible,<NA>,<NA>,provisional,<NA>,<NA>,<NA>,<NA>,<NA>
336606,2014-09-01,1552,1797,"CP Crane Power, LLC",58177,2948,Raven Power Holdings LLC,MD,S,NaT,...,<NA>,<NA>,26.09,0,final,4800977,black thunder,WY,56005,S
622681,2022-09-01,55372,4494,Harquahala Generating Project,32790,2579,Harquahala Generating Co LLC,AZ,T,NaT,...,firm,firm,<NA>,<NA>,provisional,<NA>,<NA>,<NA>,<NA>,<NA>
20555,2008-05-01,1393,517,R S Nelson,55936,4208,Entergy Gulf States Louisiana LLC,LA,C,2010-12-01,...,<NA>,<NA>,<NA>,<NA>,final,4801034,caballo mine,WY,56005,S
296254,2013-07-01,56808,603,Virginia City Hybrid Energy Center,19876,349,Dominion Virginia Power,VA,S,NaT,...,<NA>,<NA>,<NA>,<NA>,final,4407233,gobco 8,VA,<NA>,S
541208,2020-06-01,6009,632,White Bluff,814,106,Entergy Arkansas LLC,AR,C,2020-12-01,...,<NA>,<NA>,28.39,0,final,4801353,north antelope rochelle mine,WY,56005,S
556462,2020-11-01,8102,1121,Gen J M Gavin,61135,5533,"Gavin Power, LLC",OH,C,2021-12-01,...,<NA>,<NA>,5.97,0,final,4601436,shoemaker mine,WV,54051,U
195283,2011-04-01,3291,468,Parr GT,17539,292,South Carolina Electric&Gas Co,SC,C,NaT,...,interruptible,<NA>,<NA>,<NA>,final,<NA>,<NA>,<NA>,<NA>,<NA>


In [5]:
%%time
colo_df1 = pd.read_parquet(
    "/kaggle/input/catalyst-cooperative-pudl/hourly_emissions_epacems.parquet",
    dtype_backend="pyarrow",
    engine="pyarrow",
    filters=[("state", "==", "CO")],
)
colo_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13631472 entries, 0 to 13631471
Data columns (total 16 columns):
 #   Column                     Dtype                                                       
---  ------                     -----                                                       
 0   plant_id_eia               int32[pyarrow]                                              
 1   plant_id_epa               int32[pyarrow]                                              
 2   emissions_unit_id_epa      string[pyarrow]                                             
 3   operating_datetime_utc     timestamp[ms, tz=UTC][pyarrow]                              
 4   year                       int32[pyarrow]                                              
 5   state                      dictionary<values=string, indices=int32, ordered=0>[pyarrow]
 6   operating_time_hours       float[pyarrow]                                              
 7   gross_load_mw              float[pyarrow]  

In [6]:
colo_df1.sample(20)

,plant_id_eia,plant_id_epa,emissions_unit_id_epa,operating_datetime_utc,year,state,operating_time_hours,gross_load_mw,heat_content_mmbtu,steam_load_1000_lbs,so2_mass_lbs,so2_mass_measurement_code,nox_mass_lbs,nox_mass_measurement_code,co2_mass_tons,co2_mass_measurement_code
2832493,55645,55645,CT-01,2004-10-15 20:00:00+00:00,2004,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
933197,527,527,1,2000-06-19 12:00:00+00:00,2000,CO,1.0,89.0,952.700012,<NA>,240.300003,Measured,231.505997,Calculated,97.699997,Measured
6759271,56445,56445,CT-02,2011-05-05 14:00:00+00:00,2011,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3399972,469,469,1,2006-07-23 19:00:00+00:00,2006,CO,1.0,110.0,1169.099976,<NA>,746.200012,Measured,384.634003,Calculated,119.900002,Measured
11036267,55283,55283,2,2018-12-02 18:00:00+00:00,2018,CO,1.0,219.0,1578.300049,<NA>,0.947,Measured,47.348999,Unknown Code,93.800003,Substitute
8210140,6021,6021,C1,2014-10-06 11:00:00+00:00,2014,CO,1.0,436.0,4327.0,<NA>,286.399994,Measured,1068.769043,Unknown Code,453.799988,Substitute
3397776,469,469,1,2006-04-23 07:00:00+00:00,2006,CO,1.0,107.0,1141.599976,<NA>,902.099976,Measured,313.940002,Unknown Code,117.099998,Substitute
3224699,55207,55207,CT8,2005-07-20 18:00:00+00:00,2005,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11951940,525,525,H1,2020-12-05 19:00:00+00:00,2020,CO,1.0,123.0,1374.599976,<NA>,144.899994,Measured,63.231998,Calculated,141.0,Measured
11108268,55505,55505,BR1,2018-02-20 19:00:00+00:00,2018,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [7]:
from dask import dataframe as dd

In [8]:
%%time
colo_dd = dd.read_parquet(
    "/kaggle/input/catalyst-cooperative-pudl/hourly_emissions_epacems.parquet",
    engine="pyarrow",
    dtype_backend="pyarrow",
    filters=[("state", "==", "CO")],
)
colo_df2 = colo_dd.compute()
colo_df2.info()

/opt/conda/lib/python3.10/site-packages/dask/dataframe/io/parquet/core.py:1516: UserWarning: Sorted columns detected: ['state']
Use the `index` argument to set a sorted column as your index to create a DataFrame collection with known `divisions`.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13631472 entries, 0 to 13631471
Data columns (total 16 columns):
 #   Column                     Dtype                                                       
---  ------                     -----                                                       
 0   plant_id_eia               int32[pyarrow]                                              
 1   plant_id_epa               int32[pyarrow]                                              
 2   emissions_unit_id_epa      string                                                      
 3   operating_datetime_utc     timestamp[ms, tz=UTC][pyarrow]                              
 4   year                       int32[pyarrow]                                              
 5   state                      dictionary<values=string, indices=int32, ordered=0>[pyarrow]
 6   operating_time_hours       float[pyarrow]                                              
 7   gross_load_mw              float[pyarrow]  

In [9]:
colo_df2.sample(20)

,plant_id_eia,plant_id_epa,emissions_unit_id_epa,operating_datetime_utc,year,state,operating_time_hours,gross_load_mw,heat_content_mmbtu,steam_load_1000_lbs,so2_mass_lbs,so2_mass_measurement_code,nox_mass_lbs,nox_mass_measurement_code,co2_mass_tons,co2_mass_measurement_code
7026576,10682,10682,GT4,2011-11-21 07:00:00+00:00,2011,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7894803,56998,56998,CT02,2016-10-24 10:00:00+00:00,2016,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1021160,525,525,H1,2005-09-22 15:00:00+00:00,2005,CO,1.0,202.0,2371.199951,<NA>,183.100006,Substitute,1012.502014,Calculated,243.300003,Measured and Substitute
4023867,469,469,2,2008-07-16 10:00:00+00:00,2008,CO,1.0,83.0,1016.0,<NA>,366.100006,Measured,620.776001,Calculated,104.199997,Measured
40786,470,470,2,2022-08-28 17:00:00+00:00,2022,CO,1.0,210.0,2297.800049,<NA>,192.800003,Measured,328.584991,Calculated,231.5,Measured
1512283,478,478,3,2010-10-16 02:00:00+00:00,2010,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10905350,55283,55283,2,2019-01-09 21:00:00+00:00,2019,CO,1.0,180.0,1335.300049,<NA>,0.801,Substitute,37.388,Calculated,79.400002,Measured and Substitute
11329807,6112,6112,2,1997-06-24 14:00:00+00:00,1997,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9070275,8219,8219,2,2004-11-20 10:00:00+00:00,2004,CO,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6872177,6021,6021,C3,2011-04-07 00:00:00+00:00,2011,CO,1.0,448.0,4262.5,<NA>,319.799988,Measured,1244.650024,Calculated,447.0,Measured
